# Notebook 1

In this notebook we will showcase the fundamental functions needed to do the analysis. We are here looking at three essential parts:
1. [Embeddings](#embeddings)
2. [Steering](#steering)
3. [Steering Vector](#steering-vector)


In [1]:
# Append the path to the Functions directory

import sys
sys.path.append('../Functions')

## Functions used in this Notebook:

### From "Embeddings.py":
- [import_data](#import-data) - Import movie data from pickle file
- [get_embeddings](#get-embeddings) - Generate embeddings using transformer model
- [show_hidden_states](#show-hidden-states) - Display information about model layers
- [export_embeddings_to_pkl](#export-embeddings) - Save embeddings and model data to file
- [import_embedding_data_from_pkl](#import-embeddings) - Load embeddings and model data from file

### From "Steering_vector.py": 
- [import_feature_texts](#import-feature-texts) - Load feature texts for steering vector creation
- [get_steering_vector](#get-steering-vector) - Create steering vector from feature texts
- [export_steering_vector_to_pkl](#export-steering-vector) - Save steering vector to file
- [import_steering_vector_from_pkl](#import-steering-vector) - Load steering vector from file

### From "Steering.py":
- [get_steered_embeddings_vector](#get-steered-embeddings-vector) - Apply steering using semantic vector
- [get_steered_embeddings_neuron](#get-steered-embeddings-neuron) - Apply steering using specific neurons

## Import Functions

In [2]:
# Import necessary functions
from Embeddings import import_data, get_embeddings, show_hidden_states, export_embeddings_to_pkl, import_embedding_data_from_pkl
from Steering_vector import import_feature_texts, get_steering_vector, export_steering_vector_to_pkl, import_steering_vector_from_pkl
from Steering import get_steered_embeddings_vector, get_steered_embeddings_neuron

***
# <a id="embeddings"></a>Embeddings

The following functions are imported from the file `"Embeddings.py"`. They are used to create the embeddings, as well as other necessary outputs such as `encoded_input` and `hidden_states`. We also export and import the data for ease of use in other files and notebooks.

In [3]:
#from Embeddings import import_data, get_embeddings, show_hidden_states, export_embeddings_to_pkl, import_embedding_data_from_pkl

## <a id="import-data"></a>Data Import

Essential for the structure of the functions is to have the data in a specific format.
To import data, use the `import_data` function. This function imports from a `.pkl` file, in our example containing the IMDb top 1000 movies with their best fit genre. We will use this throughout the showcase.
This file needs to have three columns that contain:
- A title
- An overview of the text (this is the text we will use for embeddings)
- A label

These columns need to be stored in a `DataFrame` with the names `'title'`, `'overview'`, and `'genre'` respectively. You can have other columns as well but these will not be used. These are stored in `all_texts_data`. Change the function `import_data` in `"Embeddings.py"` if needed.

If you want to use a different model, you will perhaps need to change the architecture of the files `"Embeddings.py"` and `"Steering.py"` to fit the model.
Here we use the `all-MiniLM-L12-v2` model, which is a good choice for many tasks.

## <a id="get-embeddings"></a>Get Embeddings and <a id="show-hidden-states"></a>Show Hidden States

Below we demonstrate how to get embeddings and display hidden state information.
The `get_embeddings` function  takes in the `model name` and `overview` column returns:
- `model:` the loaded Transformer (e.g. `AutoModel`) configured for inference
- `tokenizer:` the matching tokenizer used to turn `texts` into tensors
- `model_output:` the raw `ModelOutput`(e.g. Hugging Face) with fields like `last_hidden_state`, and - because `output_hidden_states=True`- `hidden_states`
- `embeddings:` a `(batch_size, hidden_size)`tensor of sentence embeddings from mean-pooling the token embeddings, optionally L2-normalized when `normalize=true`
- `encoded_input:` the tokenzied inputs (dict with `input_ids`, `attention_mask`, and possibly `token_type_ids`) as PyTorch tensors
- `hidden_states:` a tuple of length `num_layers + 1`; element `0`is the embedding layer, and elements `1..num_layers`are the per-layer outputs. Each has shape `(batch_size, seq_len, hidden_size)`.

`show_hidden_states` simply prints out a description of the hidden states 

`all_texts_data` will be frequently used, and is the imported dataframe with at least the columns `title`, `genre` and `overview`.

In [4]:
# Import data for a chosen number of texts from the pkl file
all_texts_data = import_data(file_path="../imdb_top_1000_with_best_fit_genre.pkl", number_of_texts=1000)
texts = all_texts_data['overview'].tolist()[:1000]  # Use the overview column for text data

# Choose a model
model_name = "sentence-transformers/all-MiniLM-L12-v2"

# Get the embeddings and other model outputs
model, tokenizer, model_output, embeddings, encoded_input, hidden_states = get_embeddings(model_name, texts)

# Show hidden states information
show_hidden_states(hidden_states)

Data imported successfully.
Number of texts: 1000
                                         Poster_Link  \
0  https://m.media-amazon.com/images/M/MV5BMDFkYT...   
1  https://m.media-amazon.com/images/M/MV5BM2MyNj...   
2  https://m.media-amazon.com/images/M/MV5BMTMxNT...   
3  https://m.media-amazon.com/images/M/MV5BMWMwMG...   
4  https://m.media-amazon.com/images/M/MV5BMWU4N2...   

                      title Released_Year Certificate  Runtime  IMDB_Rating  \
0  The Shawshank Redemption          1994           A  142 min          9.3   
1             The Godfather          1972           A  175 min          9.2   
2           The Dark Knight          2008          UA  152 min          9.0   
3    The Godfather: Part II          1974           A  202 min          9.0   
4              12 Angry Men          1957           U   96 min          9.0   

                                            overview  Meta_score  \
0  Two imprisoned men bond over a number of years...        80.0   
1 

## <a id="export-embeddings"></a>Export Embeddings

All necessary data from `get_embeddings` can now be exported to a pkl file using the function `export_embeddings_to_pkl`, the parameters will be dumped to the pickle file.

In [5]:
export_embeddings_to_pkl(
    model_name, 
    model, 
    tokenizer, 
    model_output, 
    embeddings, 
    encoded_input, 
    hidden_states, 
    all_texts_data, 
    file_path="Test_export_embeddings.pkl" # Choose a filepath to save the embeddings
    ) 


Embeddings exported to Test_export_embeddings.pkl


## <a id="import-embeddings"></a>Import Embeddings

`import_embedding_data_from_pkl` allows you to import all necessary data from the exported pkl file. 
By default, no data will be imported, but you can set the parameters to `wanted_data=True` to import specific data.
Only the specified data will be returned by the function. 
The parameters must be called in the same order as they are defined in the function, shown below.

In [6]:
# Here we import all data from the exported pkl file
import_embedding_data_from_pkl("Test_export_embeddings.pkl", # Filepath to the pkl file
                               model_name=True, 
                               model=True, 
                               tokenizer=True, 
                               model_output=True, 
                               embeddings=True, 
                               encoded_input=True, 
                               hidden_states=True, 
                               all_texts_data=True);

Importing 1.36 GB data from file Test_export_embeddings.pkl...
Data imported from Test_export_embeddings.pkl
Model name: sentence-transformers/all-MiniLM-L12-v2
Model loaded successfully.
Tokenizer loaded successfully.
Model output loaded successfully.
Embeddings loaded successfully.
Encoded input loaded successfully.
Hidden states loaded successfully.
All texts data loaded successfully.
Data imported from Test_export_embeddings.pkl
Model name: sentence-transformers/all-MiniLM-L12-v2
Model loaded successfully.
Tokenizer loaded successfully.
Model output loaded successfully.
Embeddings loaded successfully.
Encoded input loaded successfully.
Hidden states loaded successfully.
All texts data loaded successfully.


Here is another example of how to use the `import_embedding_data_from_pkl` function, only importing some of the data.

In [7]:
model, embeddings, hidden_states = import_embedding_data_from_pkl("Test_export_embeddings.pkl", model=True, embeddings=True, hidden_states=True)

Importing 1.36 GB data from file Test_export_embeddings.pkl...
Data imported from Test_export_embeddings.pkl
Model loaded successfully.
Embeddings loaded successfully.
Hidden states loaded successfully.
Data imported from Test_export_embeddings.pkl
Model loaded successfully.
Embeddings loaded successfully.
Hidden states loaded successfully.


## Checkpoint
This cell will verify that everything has gone accodring to plan

In [8]:
# 🎯 CHECKPOINT 1: Embeddings Pipeline Validation
print("="*60)
print("📋 EMBEDDINGS CHECKPOINT")
print("="*60)

try:
    # Verify data loading
    print(f"✅ Data loaded: {len(all_texts_data)} movies")
    print(f"✅ Sample titles: {all_texts_data['title'].head(3).tolist()}")
    print(f"✅ Required columns present: {list(all_texts_data.columns)}")
    
    # Verify model and embeddings
    print(f"✅ Model loaded: {model_name}")
    print(f"✅ Embeddings shape: {embeddings.shape}")
    print(f"✅ Hidden states: {len(hidden_states)} layers")
    print(f"✅ Encoded input keys: {list(encoded_input.keys())}")
    
    # Memory usage info
    import os
    if os.path.exists("Test_export_embeddings.pkl"):
        file_size = os.path.getsize("Test_export_embeddings.pkl") / (1024*1024)
        print(f"✅ Export file created: Test_export_embeddings.pkl ({file_size:.1f} MB)")
    
    # Sample data validation
    print(f"✅ Sample embedding values: {embeddings[0][:3].tolist()}")
    print(f"✅ Embedding dimensions: {embeddings.shape[1]}")
    
    print("="*60)
    print("🎯 CHECKPOINT PASSED - Ready for Steering Vector creation!")
    print("="*60)
    
except Exception as e:
    print("❌ CHECKPOINT FAILED")
    print(f"💥 Error: {str(e)}")
    print("🔧 Please check previous cells and re-run if necessary")

📋 EMBEDDINGS CHECKPOINT
✅ Data loaded: 1000 movies
✅ Sample titles: ['The Shawshank Redemption', 'The Godfather', 'The Dark Knight']
✅ Required columns present: ['Poster_Link', 'title', 'Released_Year', 'Certificate', 'Runtime', 'IMDB_Rating', 'overview', 'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross', 'genre']
✅ Model loaded: sentence-transformers/all-MiniLM-L12-v2
✅ Embeddings shape: torch.Size([1000, 384])
✅ Hidden states: 13 layers
✅ Encoded input keys: ['input_ids', 'token_type_ids', 'attention_mask']
✅ Export file created: Test_export_embeddings.pkl (1391.4 MB)
✅ Sample embedding values: [-0.06406020373106003, 0.055750492960214615, -0.051509786397218704]
✅ Embedding dimensions: 384
🎯 CHECKPOINT PASSED - Ready for Steering Vector creation!


***
# <a id="steering-vector"></a>Steering Vector

The following functions are imported from the file `"Steering_vector.py"`. They include all you need to create a steering vector for a selected feature. The text representing the feature is stored in a file called `"feature.txt"` in the folder "feature". In the same folder, you can add a file called `"opposite.txt"`, where text representing the opposite of the feature is stored.

We again export the steering vectors to a pickle file so we do not need to create them multiple times. The file `"steering_vector.pkl"` can hold multiple features with the steering vector in multiple layers in a `dictionary`. To use the steering vector later, you can use the import function.

In [9]:
#from Steering_vector import import_feature_texts, get_steering_vector, export_steering_vector_to_pkl, import_steering_vector_from_pkl

## <a id="import-feature-texts"></a>Import Feature Texts and <a id="get-steering-vector"></a>Get Steering Vector

Below we demonstrate importing feature texts and creating steering vectors.

`import_feature_texts` reads the files `"feature.txt"` and `"opposite.txt"` stored in the folder named after the featre, e.g. War / Love. If the file is not present, it returns `None`. 

`get_steering_vector` creates the steering vector based on the feature text by embedding each line in the `.txt` files, before finding the mean embedding of these. 
- `layer_to_steer` decides what layer of the model outputs hidden states will be used for mean pooling. 
- You can also choose to normalize by setting `normalize=True`. We reccomend doing this as the steering vector should represent the direction of the feature, while the length/strength can be adjusted using a steering coefficient.

In [10]:
# Import feature texts
feature = "Love" # Choose a feature from the features directory.
layer_to_steer = 11 # Choose layer to create a steering vector and export for.

# Import feature texts and opposite feature texts
feature_texts, opposite_feature_texts = import_feature_texts(f"../Functions/Features/{feature}")

# Compute the steering vector
model_name = "sentence-transformers/all-MiniLM-L12-v2"
steering_vector = get_steering_vector(model_name, feature_texts, layer_to_steer=layer_to_steer, opposite_texts=opposite_feature_texts, normalize=True)

Opposite file not found: ../Functions/Features/Love/opposite.txt
Model and tokenizer set for sentence-transformers/all-MiniLM-L12-v2
Got embeddings for 25 texts using model: sentence-transformers/all-MiniLM-L12-v2
Model and tokenizer set for sentence-transformers/all-MiniLM-L12-v2
Got embeddings for 25 texts using model: sentence-transformers/all-MiniLM-L12-v2


## <a id="export-steering-vector"></a>Export Steering Vector

We can now export the steering vector for later use using the function `export_steering_vector_to_pkl`. 
- `steering_vector` is the mean embedding we got from `get_steering_vector`
- `file_path` is the path of the exported file
- `feature_name` is what feature was used to create the steering vector to keep the file organised in a dictionary
- `layer_to_steer` is what layer was steered when creating the steering vector, this also stored in the dictionary

If you want to export the steering vector for all layers, simply loop through the functions `get_steering_vector` and `export_steering_vector_to_pkl` for each layer. We reccommend doing this to begin with, so you can analyze all layers without needing
to create a new steering vector each time.


In [11]:
# Export the steering vector to a pkl file
export_steering_vector_to_pkl(steering_vector, "steering_vector.pkl", feature_name=feature, layer_to_steer=layer_to_steer)

Steering vector for 'Love' (layer 11) exported to steering_vector.pkl


In [12]:
# Loop through all layers to create and export steering vectors for each layer. You will perhaps get a too many requests error, consider 
# Running two and two features at a time.
'''
for feature in ["War", "Love", "Norway", "Baseline"]:
    feature_texts, opposite_feature_texts = import_feature_texts(f"../Functions/Features/{feature}")
    for i in range(12):
        steering_vector = get_steering_vector(model_name, feature_texts, layer_to_steer=i, opposite_texts=opposite_feature_texts, normalize=True)
        export_steering_vector_to_pkl(steering_vector, "steering_vector.pkl", feature_name=feature, layer_to_steer=i)
'''

'\nfor feature in ["War", "Love", "Norway", "Baseline"]:\n    feature_texts, opposite_feature_texts = import_feature_texts(f"../Functions/Features/{feature}")\n    for i in range(12):\n        steering_vector = get_steering_vector(model_name, feature_texts, layer_to_steer=i, opposite_texts=opposite_feature_texts, normalize=True)\n        export_steering_vector_to_pkl(steering_vector, "steering_vector.pkl", feature_name=feature, layer_to_steer=i)\n'

## <a id="import-steering-vector"></a>Import Steering Vector

`import_steering_vector_from_pkl` allows you to import a selected steering vector

- `file_path` is the path to the steering vector file
- `feature_name` is the name of the wanted feature for the steering vector
- `layer_to_steer` is what layer to retrieve the steering vector for (what layer to steer)

Note: If `layer_to_steer=None`, the function will return the steering vectors for all layers for that feature as a dictionary with the structure: `{layer: vector_tensor, layer2: vector_tensor2, ...}`

In [13]:
# This is how to import the steering vector from the pkl file
imported_steering_vector = import_steering_vector_from_pkl("steering_vector.pkl", feature_name="War", layer_to_steer=11)

Steering vectors imported from steering_vector.pkl
Available steering vectors: 'War' (layers: [10, 11]), 'Love' (layers: [11])
Returning steering vector for 'War' layer 11


## Checkpoint
This cell will verify that everything has gone accodring to plan

In [14]:
# 🎯 CHECKPOINT 2: Steering Vector Pipeline Validation
print("="*60)
print("📋 STEERING VECTOR CHECKPOINT")
print("="*60)

try:
    # Verify feature texts loading
    print(f"✅ Feature selected: '{feature}'")
    print(f"✅ Feature texts loaded: {len(feature_texts) if feature_texts else 0} lines")
    if opposite_feature_texts:
        print(f"✅ Opposite texts loaded: {len(opposite_feature_texts)} lines")
    else:
        print("ℹ️ No opposite texts found (optional)")
    
    # Verify steering vector creation
    import torch
    print(f"✅ Steering vector created for layer {layer_to_steer}")
    print(f"✅ Steering vector shape: {steering_vector.shape}")
    print(f"✅ Vector norm: {torch.norm(steering_vector):.4f}")
    print(f"✅ Is normalized: {torch.allclose(torch.norm(steering_vector), torch.tensor(1.0))}")
    
    # Verify export file
    import os
    if os.path.exists("steering_vector.pkl"):
        file_size = os.path.getsize("steering_vector.pkl") / 1024  # KB
        print(f"✅ Steering vector exported: steering_vector.pkl ({file_size:.1f} KB)")
    
    # Verify import functionality
    test_import = import_steering_vector_from_pkl("steering_vector.pkl", feature_name=feature, layer_to_steer=layer_to_steer)
    print(f"✅ Import verification: {test_import.shape}")
    
    # Sample vector values
    print(f"✅ Sample vector values: {steering_vector[:3].tolist()}")
    
    print("="*60)
    print("🎯 CHECKPOINT PASSED - Ready for Steering operations!")
    print("="*60)
    
except Exception as e:
    print("❌ CHECKPOINT FAILED")
    print(f"💥 Error: {str(e)}")
    print("🔧 Please check previous cells and re-run if necessary")
    print("💡 Tip: Ensure feature texts exist in ../Functions/Features/{feature}/")

📋 STEERING VECTOR CHECKPOINT
✅ Feature selected: 'Love'
✅ Feature texts loaded: 25 lines
ℹ️ No opposite texts found (optional)
✅ Steering vector created for layer 11
✅ Steering vector shape: torch.Size([384])
✅ Vector norm: 1.0000
✅ Is normalized: True
✅ Steering vector exported: steering_vector.pkl (5.5 KB)
Steering vectors imported from steering_vector.pkl
Available steering vectors: 'War' (layers: [10, 11]), 'Love' (layers: [11])
Returning steering vector for 'Love' layer 11
✅ Import verification: torch.Size([384])
✅ Sample vector values: [0.00398731604218483, 0.07698136568069458, 0.027998507022857666]
🎯 CHECKPOINT PASSED - Ready for Steering operations!


***
# <a id="steering"></a>Steering

The following functions are imported from the file `"Steering.py"` and include what you need to create the steered embeddings and outputs using either the steering vector, or chosing to steer on a specific node in a specific layer. We will reuse the function `import_embedding_data_from_pkl` that we saw under Embeddings, and the `import_steering_vector_from_pkl` we saw under Steering Vector.

In [15]:
#from Steering import get_steered_embeddings_vector, get_steered_embeddings_neuron

## <a id="get-steered-embeddings-vector"></a>Get Steered Embeddings (Vector)

Below we demonstrate steering using a semantic vector:

`get_steered_embeddings_vector` creates steered embeddings using the steering vector. You must first import the `model` and `encoded_input` you used for the original embeddings.

- `layer_to_steer` is what layer you are wanting to steer. This should be the same as the layer of the imported steering_vector. If no layer is specefied for the steering vector, the function will use this parameter to choose what layer to use as the steering vector.
- `steering_coefficient` is the coefficient which you multiply the steering vector with in the steering hook
- `steering_vector` is the chosen steering vector. This can be a dictionary of all the layers with the respective vector, or a single vector
- `normalize` will normalize the embeddings

In [16]:
from Steering import get_steered_embeddings_vector, get_steered_embeddings_neuron

In [17]:
# Import data
model, encoded_input = import_embedding_data_from_pkl('Test_export_embeddings.pkl', model=True, encoded_input=True)
steering_vector = import_steering_vector_from_pkl('steering_vector.pkl', 'War') # If no layer is specified, all vectors are returned

Importing 1.36 GB data from file Test_export_embeddings.pkl...
Data imported from Test_export_embeddings.pkl
Model loaded successfully.
Encoded input loaded successfully.
Steering vectors imported from steering_vector.pkl
Available steering vectors: 'War' (layers: [10, 11]), 'Love' (layers: [11])
Returning all layers for 'War': [10, 11]
Data imported from Test_export_embeddings.pkl
Model loaded successfully.
Encoded input loaded successfully.
Steering vectors imported from steering_vector.pkl
Available steering vectors: 'War' (layers: [10, 11]), 'Love' (layers: [11])
Returning all layers for 'War': [10, 11]


In [18]:
# Here we compute the steered embeddings using the steering vector from a specific layer.

steered_embeddings_vector = get_steered_embeddings_vector(
    model, 
    encoded_input, 
    layer_to_steer=11, 
    steering_coefficient=0.5, 
    steering_vector=steering_vector, # If all vectors are passed, only the selected layer is used
    normalize=True
)

Created steered model output with shape: torch.Size([1000, 66, 384])
Created steered embeddings with shape: torch.Size([1000, 384])


## <a id="get-steered-embeddings-neuron"></a>Get Steered Embeddings (Neuron)

`get_steered_embeddings_neuron` can be used to create steered embeddings when steering on a single neuron in a specified layer.
- `layer_to_steer` is in what layer you want to steer your neuron
- `node_to_steer` is the neuron index of the neuron you want to steer. This can be either `int`or `tuple`, where a tuple will steer on the neurons listed.
- `steering_coefficient` is the number you will add to the output in the steering hook. This can be either `int`or `tuple`, where a tuple will steer the corresponding indexed neurons in `node_to_steer` by that coefficient.
- `normalize` will normalize the embeddings

In [19]:
# Here we compute the steered embeddings using a specific neuron in a specific layer.

steered_embeddings_neuron = get_steered_embeddings_neuron(
    model, 
    encoded_input, 
    layer_to_steer=11,
    node_to_steer=0, 
    steering_coefficient=0.5,
    normalize=True
    )

Created steered model output with shape: torch.Size([1000, 66, 384])
Created steered embeddings with shape: torch.Size([1000, 384])


If you want to steer on multiple neurons, you can pass a `tuple` of neuron indices and a tuple of steering coefficients. 
This will steer neuron 0 with a coefficient of 0.5, neuron 42 with a coefficient of 1, and neuron 100 with a coefficient of 2.

In [20]:
steered_embeddings_neurons = get_steered_embeddings_neuron(
    model, 
    encoded_input, 
    layer_to_steer=11, 
    node_to_steer=(0, 42, 100),
    steering_coefficient=(0.5, 1, 2), 
    normalize=True
)

Created steered model output with shape: torch.Size([1000, 66, 384])
Created steered embeddings with shape: torch.Size([1000, 384])


## Checkpoint
This cell will verify that everything has gone accodring to plan

In [ ]:
# 🎯 CHECKPOINT 3: Steering Operations Validation
print("="*60)
print("📋 STEERING OPERATIONS CHECKPOINT")
print("="*60)

try:
    # Verify data imports for steering
    print(f"✅ Model and encoded_input loaded for steering")
    print(f"✅ Steering vector imported for 'War' feature")
    
    # Verify vector steering
    print(f"✅ Vector steering completed")
    print(f"✅ Steered embeddings (vector) shape: {steered_embeddings_vector.shape}")
    print(f"✅ Steering applied to layer 11 with coefficient 0.5")
    
    # Verify neuron steering (single)
    print(f"✅ Single neuron steering completed")
    print(f"✅ Steered embeddings (neuron) shape: {steered_embeddings_neuron.shape}")
    print(f"✅ Neuron 0 steered with coefficient 0.5")
    
    # Verify multi-neuron steering
    print(f"✅ Multi-neuron steering completed")
    print(f"✅ Steered embeddings (neurons) shape: {steered_embeddings_neurons.shape}")
    print(f"✅ Neurons (0, 42, 100) steered with coefficients (0.5, 1, 2)")
    
    # Compare original vs steered embeddings
    original_sample = embeddings[0][:3]
    steered_vector_sample = steered_embeddings_vector[0][:3]
    steered_neuron_sample = steered_embeddings_neuron[0][:3]
    
    import torch
    vector_diff = torch.norm(steered_vector_sample - original_sample)
    neuron_diff = torch.norm(steered_neuron_sample - original_sample)
    
    print(f"✅ Original sample: {original_sample.tolist()}")
    print(f"✅ Vector steered sample: {steered_vector_sample.tolist()}")
    print(f"✅ Vector steering magnitude: {vector_diff:.4f}")
    print(f"✅ Neuron steering magnitude: {neuron_diff:.4f}")
    
    print("="*60)
    print("🎯 ALL CHECKPOINTS PASSED - Analysis pipeline complete!")
    print("🚀 Ready to move on to Notebook 2 (PCA, t-SNE, ranking, etc.)")
    print("="*60)
    
except Exception as e:
    print("❌ CHECKPOINT FAILED")
    print(f"💥 Error: {str(e)}")
    print("🔧 Please check previous cells and re-run if necessary")
    print("💡 Tip: Ensure all previous checkpoints passed successfully")

📋 STEERING OPERATIONS CHECKPOINT
✅ Model and encoded_input loaded for steering
✅ Steering vector imported for 'War' feature
✅ Vector steering completed
✅ Steered embeddings (vector) shape: torch.Size([1000, 384])
✅ Steering applied to layer 11 with coefficient 0.5
✅ Single neuron steering completed
✅ Steered embeddings (neuron) shape: torch.Size([1000, 384])
✅ Neuron 0 steered with coefficient 0.5
✅ Multi-neuron steering completed
✅ Steered embeddings (neurons) shape: torch.Size([1000, 384])
✅ Neurons (0, 42, 100) steered with coefficients (0.5, 1, 2)
✅ Original sample: [-0.06406020373106003, 0.055750492960214615, -0.051509786397218704]
✅ Vector steered sample: [-0.06847507506608963, 0.05591105297207832, -0.06437089294195175]
✅ Vector steering magnitude: 0.0136
✅ Neuron steering magnitude: 0.1220
🎯 ALL CHECKPOINTS PASSED - Analysis pipeline complete!
🚀 Ready for downstream analysis (PCA, t-SNE, ranking, etc.)
